In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

In [17]:
inputs = Input(shape=(28, 28), name="input_layer")
x = Flatten(name="flatten_layer")(inputs)   
h1 = Dense(16, activation='relu', name="hidden_layer_1")(x)
h2 = Dense(16, activation='relu', name="hidden_layer_2")(h1)
h3 = Dense(16, activation='relu', name="hidden_layer_3")(h2)
outputs = Dense(10, activation='softmax', name="output_layer")(h3)
model = Model(inputs=inputs, outputs=outputs, name="mnist_model")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [18]:
from tensorflow.keras.datasets import mnist

# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize
x_train = x_train / 255.0
x_test = x_test / 255.0
# Train
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
model.save('my_model.keras')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8481 - loss: 0.5017 - val_accuracy: 0.9106 - val_loss: 0.3056
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9208 - loss: 0.2722 - val_accuracy: 0.9266 - val_loss: 0.2471
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9342 - loss: 0.2281 - val_accuracy: 0.9288 - val_loss: 0.2392
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9410 - loss: 0.1986 - val_accuracy: 0.9433 - val_loss: 0.2020
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9469 - loss: 0.1797 - val_accuracy: 0.9462 - val_loss: 0.1875


In [1]:
import tkinter as tk
from PIL import Image, ImageDraw
import numpy as np
import tensorflow as tf

# ------------------------
# Load your trained model
# ------------------------
model = tf.keras.models.load_model("my_model.keras")

# ------------------------
# GUI Settings
# ------------------------
WIDTH, HEIGHT = 280, 280   # Drawing canvas size
GRID_SIZE = 28             # Model input size
BG_COLOR = "white"         # Canvas background
DRAW_COLOR = "black"       # Drawing color
BRUSH_SIZE = 10            # Brush radius

# ------------------------
# Create Tkinter Window
# ------------------------
root = tk.Tk()
root.title("Digit Recognizer (28x28 FCNN)")

# Create Canvas
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg=BG_COLOR, cursor="cross")
canvas.grid(row=0, column=0, pady=2, sticky=tk.W, columnspan=2)

# Create a Pillow image to mirror what’s drawn on canvas
image = Image.new("L", (WIDTH, HEIGHT), color=255)  # white background
draw = ImageDraw.Draw(image)

# ------------------------
# Drawing Function
# ------------------------
def paint(event):
    x1, y1 = (event.x - BRUSH_SIZE), (event.y - BRUSH_SIZE)
    x2, y2 = (event.x + BRUSH_SIZE), (event.y + BRUSH_SIZE)
    canvas.create_oval(x1, y1, x2, y2, fill=DRAW_COLOR, outline=DRAW_COLOR)
    draw.ellipse([x1, y1, x2, y2], fill=0)  # black ink in Pillow image

canvas.bind("<B1-Motion>", paint)

# ------------------------
# Prediction Function
# ------------------------
def predict_digit():
    # Resize to 28x28
    img_resized = image.resize((GRID_SIZE, GRID_SIZE))
    img_array = np.array(img_resized)
    img_array = 255 - img_array        # invert colors: white bg → 0, black ink → 255
    img_array = img_array / 255.0      # normalize
    img_array = img_array.reshape(1, 28, 28)  # FCNN input shape

    # Predict
    prediction = model.predict(img_array)
    digit = np.argmax(prediction)
    confidence = np.max(prediction) * 100

    result_label.config(text=f"Prediction: {digit} ({confidence:.2f}%)")

# ------------------------
# Clear Function
# ------------------------
def clear_canvas():
    canvas.delete("all")
    draw.rectangle([0, 0, WIDTH, HEIGHT], fill=255)
    result_label.config(text="Prediction: None")

# ------------------------
# Buttons & Label
# ------------------------
predict_button = tk.Button(root, text="Predict", command=predict_digit)
predict_button.grid(row=1, column=0, pady=2)

clear_button = tk.Button(root, text="Clear", command=clear_canvas)
clear_button.grid(row=1, column=1, pady=2)

result_label = tk.Label(root, text="Prediction: None", font=("Helvetica", 16))
result_label.grid(row=2, column=0, columnspan=2)

root.mainloop()


2025-08-10 00:02:39.798212: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-10 00:02:39.861027: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-10 00:02:40.199262: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-10 00:02:40.531007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754762560.998974   42952 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754762561.13

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
